In [1]:
from ber_metrics import BEREstimator
import numpy as np
import pickle
import pandas as pd


## Adult Income

In [2]:
X_train, y_train, X_test, y_test = pickle.load(open('Data/adult_income/processed_data.pkl', 'rb'))
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
print(X_train.shape)
adult_estimator = BEREstimator(X_train, y_train)
print("Bhattacharrya upper bound: {}".format(adult_estimator.bhattacharyya_bound()))
print("Mahalanobis upper bound: {}".format(adult_estimator.mahalanobis_bound()))
print("Nearest Neighbor upper bound: {}".format(adult_estimator.nn_bound(leafsize=29000)))


(30162, 104)
> /home/matthewkol186/pton-research/cos597e/cos597efinalproject/ber_metrics.py(87)bhattacharyya_bound()
-> return np.exp(-first_term-second_term-third_term) * np.sqrt(p_0 * p_1) # for now, only interested in upper bound


(Pdb)  -first_term-second-term-third_term


*** NameError: name 'second' is not defined


(Pdb)  -first_term-second_term-third_term


-88.87818740183798


(Pdb)  q


BdbQuit: 

In [9]:
X_train.shape

(30162, 104)

## COMPAS Recidivism

In [3]:
X_train, y_train, X_test, y_test = pickle.load(open('Data/compas/processed_violent.pkl', 'rb'))
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
violent_estimator = BEREstimator(X_train, y_train)
print("Bhattacharrya upper bound: {}".format(violent_estimator.bhattacharyya_bound()))
print("Mahalanobis upper bound: {}".format(violent_estimator.mahalanobis_bound()))
print("Nearest Neighbor upper bound: {}".format(violent_estimator.nn_bound()))

> /home/matthewkol186/pton-research/cos597e/cos597efinalproject/ber_metrics.py(87)bhattacharyya_bound()
-> return np.exp(-first_term-second_term-third_term) * np.sqrt(p_0 * p_1) # for now, only interested in upper bound


(Pdb)  c


Bhattacharrya upper bound: 0.02596945557312589
Mahalanobis upper bound: 0.18821591060670279
Nearest Neighbor upper bound: 0.16203143893591293


In [6]:
X_train, y_train, X_test, y_test = pickle.load(open('Data/compas/processed_arrest.pkl', 'rb'))
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
violent_estimator = BEREstimator(X_train, y_train)
print("Bhattacharrya upper bound: {}".format(violent_estimator.bhattacharyya_bound()))
print("Mahalanobis upper bound: {}".format(violent_estimator.mahalanobis_bound()))
print("Nearest Neighbor upper bound: {}".format(violent_estimator.nn_bound()))

Bhattacharrya upper bound: 1.343260226887181e-08
Mahalanobis upper bound: 0.4158120026746693
Nearest Neighbor upper bound: 0.441596130592503
